In [1]:
#---------------------------------------------------------------------------
#                ___________
#    ______     /   ____    ____      _          ______
#   |  ____|   /   /    INRS    \    | |        |  ____|
#   | |       /   /     Edge     \   | |        | |
#   | |____  /   /    Computing   \  | |        | |____
#   |  ____| \   \  Communication /  | |        |  ____|   
#   | |       \   \   Learning   /   | |        | |
#   | |____    \   \_____LAB____/    | |_____   | |____
#   |______|    \ ___________        |_______|  |______|
#
#  Edge Computing, Communication and Learning Lab - INRS University
#
#  Author: Mobin Vaziri
#
#  Project: HENNC
#  
#  Creation Date: 2023-07-08
#
#  Description:  
#---------------------------------------------------------------------------

### Import Packages

In [38]:
import numpy as np
import os

# Run Keras on GPU
os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.models import load_model
from scipy.integrate import odeint
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
import pandas as pd
import matplotlib.pyplot as plt
import random as python_random
import pandas as pd


### Create  The Dataset

In [15]:
# Define the Chen system
def chen_system(state, t):
    x, y, z = state  # Unpack the state vector
    a = 40
    b = 3
    c = 28
    dx = a * (y - x)
    dy = (c - a) * x - x * z + c * y
    dz = x * y - b * z
    return dx, dy, dz  # Derivatives

# Number of points
num_points = 100000  # Adjust this to change the number of points

# Solve the system
state0 = [-0.1, 0.5, -0.6]
t = np.linspace(0.0, 100.0, num_points)
states = odeint(chen_system, state0, t)

# Create the dataset
X = states[:-1]
Y = states[1:]

# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

#input data normalization
scaler = StandardScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)

X_test_scaled = scaler.transform(X_test)

### Fixing the seeds

In [16]:
def reset_seeds():
   
   np.random.seed(42) 
   python_random.seed(42)
   tf.random.set_seed(42)

### ANN Model

In [21]:
reset_seeds()

# Define the MLP
model = Sequential()

model.add(Dense(8, input_dim = 3,  name="fc_1"))

model.add(Activation(activation = 'relu', name = "act_1"))

model.add(Dense(3, name = "fc_2"))

model.add(Activation(activation = "linear", name = "act_2"))

# Optimizer
opt = keras.optimizers.Adam(learning_rate=0.0001)

# Loss Function
loss_fn = keras.losses.MeanSquaredError()

# Compile the model
model.compile(loss = loss_fn, optimizer = opt)

# Early Stopping
callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3)


model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 fc_1 (Dense)                (None, 8)                 32        
                                                                 
 act_1 (Activation)          (None, 8)                 0         
                                                                 
 fc_2 (Dense)                (None, 3)                 27        
                                                                 
 act_2 (Activation)          (None, 3)                 0         
                                                                 
Total params: 59
Trainable params: 59
Non-trainable params: 0
_________________________________________________________________


In [43]:
# If true the pretrained model will be used else the model is going to be trained.
pre_trained = True

In [44]:
if pre_trained == True:

    model = load_model("my_model.h5")

else:
    # Train the model and store the history
    history = model.fit(X_train_scaled, Y_train, validation_data=(X_test_scaled, Y_test), epochs = 300, batch_size=128, verbose=1, callbacks=[callback])

    # Plot MSE vs epoch
    plt.figure()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model RMSE')
    plt.ylabel('RMSE')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper right')
    plt.show()


### Accuracy Measurement

In [45]:
# Run inference on the training and testing sets
Y_train_pred = model.predict(X_train_scaled)
Y_test_pred = model.predict(X_test_scaled)

# Calculate MSE
train_mse = mean_squared_error(Y_train, Y_train_pred)
test_mse = mean_squared_error(Y_test, Y_test_pred)

# Calculate MAE
train_mae = mean_absolute_error(Y_train, Y_train_pred)
test_mae = mean_absolute_error(Y_test, Y_test_pred)

# Calculate RMSE
train_rmse = sqrt(train_mse)
test_rmse = sqrt(test_mse)

# Calculate R2 Score
train_r2 = r2_score(Y_train, Y_train_pred)
test_r2 = r2_score(Y_test, Y_test_pred)


# Calculate the metrics
metrics = {
    'MSE': [train_mse, test_mse],
    'MAE': [train_mae, test_mae],
    'RMSE': [train_rmse, test_rmse],
    'R2 Score': [train_r2, test_r2]
}

# Create a DataFrame
df = pd.DataFrame(metrics, index=['Train', 'Test'])

# Print the DataFrame
print(df)

625/625 [==============================] - 0s 603us/step
            MSE       MAE      RMSE  R2 Score
Train  0.000327  0.011742  0.018095  0.999989
Test   0.000308  0.011653  0.017547  0.999990


In [40]:
output = model.predict(X_test_scaled)

625/625 [==============================] - 1s 781us/step


In [46]:
w_l1 = model.layers[0].get_weights()[0]
b_l1 = model.layers[0].get_weights()[1]

w_l2 = model.layers[2].get_weights()[0]
b_l2 = model.layers[2].get_weights()[1]

w_1 = list()
b_1 = list()

w_2 = list()
b_2 = list()

for r in range (0, w_l1.shape[0]):
    for c in range (0, w_l1.shape[1]):
        
        w_1.append(w_l1[r][c])

w_1 = np.array(w_1)

w_1 = w_1.reshape((w_l1.shape[0],w_l1.shape[1]))


for r in range (0, w_l2.shape[0]):
    for c in range (0, w_l2.shape[1]):
        
        w_2.append(w_l2[r][c])

w_2 = np.array(w_2)

w_2 = w_2.reshape((w_l2.shape[0],w_l2.shape[1]))


for i in range (0, b_l1.shape[0]):
    
    b_1.append(b_l1[i])
    
b_1 = np.array(b_1)

b_1 = b_1.reshape(b_l1.shape[0])


for i in range (0, b_l2.shape[0]):
    
    b_2.append(b_l2[i])
    
b_2 = np.array(b_2)

b_2 = b_2.reshape(b_l2.shape[0])

print("Shape of w_1 is {}, w_2 is {}, b_1 is {}, and b_2 is {}" .format(w_1.shape, w_2.shape, b_1.shape, b_2.shape))

np.savetxt('w_1.txt', w_1, delimiter=' ', fmt="%s")
np.savetxt('w_2.txt', w_2, delimiter=' ', fmt="%s")
np.savetxt('b_1.txt', b_1, delimiter=' ', fmt="%s")
np.savetxt('b_2.txt', b_2, delimiter=' ', fmt="%s")

Shape of w_1 is (3, 8), w_2 is (8, 3), b_1 is (8,), and b_2 is (3,)
